In [1]:
import os
import glob
import subprocess
import shutil
import ants
import numpy as np
from antspynet.utilities import brain_extraction
from datetime import datetime
import json

# =========================================================
# 1. 경로 설정
# =========================================================
PROJECT_DIR = '/home/oracle/preprocessing'

# 📂 두 개의 입력 폴더 리스트
INPUT_DIRS = [
    os.path.join(PROJECT_DIR, 'mri', '7team_adni_all', 'ADNI'),
    os.path.join(PROJECT_DIR, 'mri', '7team_adni_all_dataset', 'ADNI'),
]

# 결과 저장 폴더 (하나로 통합)
OUTPUT_ROOT_DIR = os.path.join(PROJECT_DIR, 'assets', 'adni_all_data_ROBUST')

# 템플릿 경로
TEMPLATE_PATH = os.path.join(PROJECT_DIR, 'assets', 'templates', 'mni_icbm152_t1_tal_nlin_sym_09a.nii')

# 로그 폴더
LOG_DIR = os.path.join(OUTPUT_ROOT_DIR, 'logs')

# =========================================================
# 2. 설정 옵션
# =========================================================
CONFIG = {
    'registration_type': 'SyNRA',
    'clip_range': (-5, 5),
}

# 폴더 생성
os.makedirs(OUTPUT_ROOT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# 템플릿 확인
print(f"📍 템플릿 경로: {TEMPLATE_PATH}")
if os.path.exists(TEMPLATE_PATH):
    print("✅ 템플릿 파일 확인!")
else:
    raise FileNotFoundError("❌ 템플릿 파일을 찾을 수 없습니다!")

# =========================================================
# 3. 핵심 함수
# =========================================================

def convert_dicom_to_nifti(dicom_folder, temp_output_dir):
    """DICOM을 NIfTI로 변환"""
    os.makedirs(temp_output_dir, exist_ok=True)
    
    cmd = f"dcm2niix -z y -f %p_%t_%s -o {temp_output_dir} {dicom_folder}"
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    return glob.glob(os.path.join(temp_output_dir, "*.nii.gz"))


def preprocess_single_image(nii_path, save_dir, template_path, subject_id):
    """
    단일 이미지 전처리
    순서: Reorient -> Bias -> Registration -> Brain Extraction -> Normalization
    """
    result = {
        'subject_id': subject_id,
        'input_file': nii_path,
        'status': 'unknown',
        'error_message': None,
        'output_file': None,
    }
    
    try:
        filename = os.path.basename(nii_path)
        final_filename = f"{subject_id}_{filename.replace('.nii.gz', '')}_preprocessed.nii.gz"
        final_path = os.path.join(save_dir, final_filename)

        # 이미 처리된 파일은 건너뛰기
        if os.path.exists(final_path):
            print(f"   ⚠️ 이미 존재 (Pass): {final_filename}")
            result['status'] = 'skipped'
            result['output_file'] = final_path
            return result

        print(f"   ▶ 작업 시작: {final_filename}")

        # 1. 템플릿 로드
        fixed_img = ants.image_read(template_path, reorient='RPI')

        # 2. 이미지 로드
        img = ants.image_read(nii_path, reorient='RPI')

        # 3. Bias Field Correction
        img_n4 = ants.n4_bias_field_correction(img)

        # 4. Registration
        mytx = ants.registration(
            fixed=fixed_img, 
            moving=img_n4, 
            type_of_transform=CONFIG['registration_type']
        )
        img_reg = mytx['warpedmovout']

        # 5. Brain Extraction
        prob_mask = brain_extraction(img_reg, modality="t1", verbose=False)
        brain_mask = ants.threshold_image(prob_mask, low_thresh=0.5, high_thresh=1.0, inval=1, outval=0)
        brain_extracted = img_reg * brain_mask

        # 6. Robust Z-Score Normalization
        brain_numpy = brain_extracted.numpy()
        mask_indices = brain_numpy != 0
        brain_pixels = brain_numpy[mask_indices]
        
        if len(brain_pixels) > 0:
            median = np.median(brain_pixels)
            q75, q25 = np.percentile(brain_pixels, [75, 25])
            iqr = q75 - q25
            normalized_pixels = (brain_pixels - median) / (iqr + 1e-7)
            brain_numpy[mask_indices] = normalized_pixels
            
            # 클리핑
            brain_numpy = np.clip(brain_numpy, CONFIG['clip_range'][0], CONFIG['clip_range'][1])
        
        brain_normalized = ants.from_numpy(
            brain_numpy, 
            origin=img_reg.origin, 
            spacing=img_reg.spacing, 
            direction=img_reg.direction
        )

        # 7. 저장
        ants.image_write(brain_normalized, final_path)
        print(f"   ✅ 저장 완료: {final_filename}")
        
        result['status'] = 'success'
        result['output_file'] = final_path
        
    except Exception as e:
        error_msg = str(e)
        print(f"   ❌ 실패 ({subject_id}): {error_msg}")
        result['status'] = 'failed'
        result['error_message'] = error_msg
    
    return result


def process_subject(subject_id, dicom_folder):
    """환자별 처리"""
    temp_dir = os.path.join(OUTPUT_ROOT_DIR, f"temp_{subject_id}")
    results = []
    
    try:
        # DICOM -> NIfTI 변환
        converted_nii_files = convert_dicom_to_nifti(dicom_folder, temp_dir)
        
        if not converted_nii_files:
            print(f"\n⚠️ {subject_id}: NIfTI 파일 없음 (Pass)")
            return results
        
        print(f"\n🔎 처리 중: {subject_id} ({len(converted_nii_files)}개 파일)")
        
        # 각 NIfTI 파일 전처리
        for nii in converted_nii_files:
            result = preprocess_single_image(nii, OUTPUT_ROOT_DIR, TEMPLATE_PATH, subject_id)
            results.append(result)
            
    finally:
        # 임시 폴더 정리
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
    
    return results


def collect_subjects(input_root_dir):
    """DICOM 폴더 수집"""
    subjects = []
    for root, dirs, files in os.walk(input_root_dir):
        if len(files) > 0:
            relative_path = os.path.relpath(root, input_root_dir)
            path_parts = relative_path.split(os.sep)
            if len(path_parts) < 1:
                continue
            subject_id = path_parts[0]
            subjects.append((subject_id, root))
    return subjects


def save_report(all_results):
    """처리 결과 리포트 저장"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    total = len(all_results)
    success = sum(1 for r in all_results if r['status'] == 'success')
    failed = sum(1 for r in all_results if r['status'] == 'failed')
    skipped = sum(1 for r in all_results if r['status'] == 'skipped')
    
    report = {
        'timestamp': timestamp,
        'summary': {
            'total': total,
            'success': success,
            'failed': failed,
            'skipped': skipped
        },
        'results': all_results
    }
    
    report_path = os.path.join(LOG_DIR, f"report_{timestamp}.json")
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    print("\n" + "="*60)
    print(f"📊 처리 결과: 총 {total}개")
    print(f"   ✅ 성공: {success}개")
    print(f"   ❌ 실패: {failed}개")
    print(f"   ⚠️ 건너뜀: {skipped}개")
    print(f"📄 리포트: {report_path}")
    print("="*60)


# =========================================================
# 4. 메인 실행
# =========================================================

if __name__ == "__main__":
    print("="*60)
    print("🧠 MRI 전처리 파이프라인 (순차 처리)")
    print("="*60)
    print(f"📋 Registration: {CONFIG['registration_type']}")
    print(f"📋 Clipping: {CONFIG['clip_range']}")
    
    # 여러 입력 폴더에서 환자 데이터 수집
    all_subjects = []
    
    for input_dir in INPUT_DIRS:
        print(f"\n📂 데이터 탐색: {input_dir}")
        
        if not os.path.exists(input_dir):
            print(f"⚠️ 폴더 없음 (건너뜀): {input_dir}")
            continue
        
        subjects = collect_subjects(input_dir)
        print(f"📊 발견된 환자 수: {len(subjects)}")
        all_subjects.extend(subjects)
    
    print(f"\n📊 총 환자 수: {len(all_subjects)}")
    
    if len(all_subjects) == 0:
        print("❌ 처리할 데이터가 없습니다!")
        exit(1)
    
    print(f"\n🚀 순차 처리 시작...\n")
    
    all_results = []
    for idx, (subject_id, dicom_folder) in enumerate(all_subjects, 1):
        print(f"[{idx}/{len(all_subjects)}]", end=" ")
        results = process_subject(subject_id, dicom_folder)
        all_results.extend(results)
    
    save_report(all_results)
    print("\n🎉 모든 작업 완료!")

2026-02-04 21:43:16.484147: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-04 21:43:16.574410: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-04 21:43:16.997323: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-02-04 21:43:16.997429: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-02-04 21:43:17.001331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

📍 템플릿 경로: /home/oracle/preprocessing/assets/templates/mni_icbm152_t1_tal_nlin_sym_09a.nii
✅ 템플릿 파일 확인!
🧠 MRI 전처리 파이프라인 (순차 처리)
📋 Registration: SyNRA
📋 Clipping: (-5, 5)

📂 데이터 탐색: /home/oracle/preprocessing/mri/7team_adni_all/ADNI
📊 발견된 환자 수: 397

📂 데이터 탐색: /home/oracle/preprocessing/mri/7team_adni_all_dataset/ADNI
📊 발견된 환자 수: 637

📊 총 환자 수: 1034

🚀 순차 처리 시작...

[1/1034] 
🔎 처리 중: 003_S_4118 (1개 파일)
   ▶ 작업 시작: 003_S_4118_223336_4_20110809082218_5_preprocessed.nii.gz


2026-02-04 21:44:12.932230: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2026-02-04 21:44:12.934058: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


   ✅ 저장 완료: 003_S_4118_223336_4_20110809082218_5_preprocessed.nii.gz
[2/1034] 
🔎 처리 중: 116_S_6775 (1개 파일)
   ▶ 작업 시작: 116_S_6775_Accelerated_Sagittal_MPRAGE_20190815133459_3_preprocessed.nii.gz
   ✅ 저장 완료: 116_S_6775_Accelerated_Sagittal_MPRAGE_20190815133459_3_preprocessed.nii.gz
[3/1034] 
🔎 처리 중: 941_S_6094 (1개 파일)
   ▶ 작업 시작: 941_S_6094_Accelerated_Sagittal_MPRAGE_20171023150525_2_preprocessed.nii.gz
   ✅ 저장 완료: 941_S_6094_Accelerated_Sagittal_MPRAGE_20171023150525_2_preprocessed.nii.gz
[4/1034] 
🔎 처리 중: 128_S_1242 (1개 파일)
   ▶ 작업 시작: 128_S_1242_MPRAGE_20070214133639_2_preprocessed.nii.gz
   ✅ 저장 완료: 128_S_1242_MPRAGE_20070214133639_2_preprocessed.nii.gz
[5/1034] 
🔎 처리 중: 052_S_6305 (1개 파일)
   ▶ 작업 시작: 052_S_6305_Accelerated_Sagittal_MPRAGE_20180504131448_2_preprocessed.nii.gz
   ✅ 저장 완료: 052_S_6305_Accelerated_Sagittal_MPRAGE_20180504131448_2_preprocessed.nii.gz
[6/1034] 
🔎 처리 중: 116_S_6550 (1개 파일)
   ▶ 작업 시작: 116_S_6550_Accelerated_Sagittal_MPRAGE_20180808143217_3_preprocessed.nii